# L22: Explicit Type Checking and Type Inference

* Look on Moodle for "Week11" no spaces and you'll easily find the folder with this document

## TODOs:
* Spot Exam 3
    * Not this Friday's Recitation! WOO! It's on 04/12
    * This will be a Moodle exam (like spot exam 2)
    * This will cover:
        * Scala Higher Order methods of Lists (map, reduce, fold...)
        * Lettuce recursive functions
            * value of a program
            * use of ExtendRec
        * References and Mutability
            * Implicit
            * Explicit
            * (sideeffects)
        * CPS transforms using the _k notation and generic types

## Overview
* Explicit Type Checking (static time)
* Review Static vs Dynamic Type Checking
* Type Inference (still static time)
* Unification

## Explicit Type Checking

###  Concrete Lettuce with Types
* We need to update the lettuce concrete syntax to allow the programmers to state types explicitly
* The powers that be have determined that the new concrete lettuce grammar is as follows (**changes in bold font**):
    * $$\begin{array} &
expr & \rightarrow & value \\
& | & x \\
& | & expr\ +\ expr \\
& | & -\ expr \\
& | & expr\ ||\ expr \\
& | & expr < expr \\
& | & !\ expr \\
& | & if\ (expr)\ then\ expr\ else\ expr \\
& | & let\ x\ \mathbf{:\ type}\ =\ expr\ in\ expr \\
& | & let\ rec\ x\ \mathbf{:\ type}\ =\ expr\ in\ expr \\
& | & expr(expr) \\
\\
value & \rightarrow & n \\
& | & b \\
& | & function\ (x \mathbf{:\ type}\ )\ expr \\
\\
 \mathbf{type} & \rightarrow &  \mathbf{num} \\
& | &  \mathbf{bool} \\
& | &  \mathbf{type\ =>\ type} \\
\\
(\ n\ is\ a\ number\ value\ ) \\
(\ b\ is\ a\ boolean\ value\ ) \\
(\ x\ is\ an\ identifier\ )
\end{array}$$

### Abstract Lettuce with Types
* Now let us note that we need to abstract concrete Lettuce to an AST
* we would typically write a generative grammar
* but I don't feel like typesetting very much of it today, so let's look at the relavant subset
    * please let me know if you'd like more detail

#### OLD: no types
$$\begin{array} &
Expr & \rightarrow & Value \\
& | & Let(Identifier,\ Expr,\ Expr) \\
& | & LetRec(Identifier_{functionName},\ Identifier_{param},\ Expr,\ Expr) \\
\\
Value & \rightarrow & FunDef(Identifier,\ Expr) \\
\\
Identifier & \rightarrow & String \\
\\
(Type\ not\ defined)
\end{array}$$

#### NEW: with types
$$\begin{array} &
Expr & \rightarrow & Value \\
& | & Let(Identifier,\ Type,\ Expr,\ Expr) \\
& | & LetRec(Identifier_{functionName},\ Type_{functionType},\ Identifier_{param},\ Type_{paramType},\ Expr,\ Expr) \\
\\
Value & \rightarrow & FunDef(Identifier,\ Type,\ Expr) \\
\\
Identifier & \rightarrow & String \\
\\
Type & \rightarrow & NumType \\
& | & BoolType \\
& | & FunType(Type_{input},\ Type_{output})\\
\end{array}$$

#### Qs???

###  Code
* So let's just jump into the code
* I'll get us started
* And then I'll ask you to explore on your own

In [15]:
// AST
sealed trait Expr
sealed trait Value extends Expr
sealed trait Type
type Identifier = String
type Number = Double
type TypeEnv = Map[Identifier,Type]

case class Ident(x:Identifier) extends Expr
case class Plus(e1:Expr, e2:Expr) extends Expr
case class Minus(e1:Expr, e2:Expr) extends Expr
case class Times(e1:Expr, e2:Expr) extends Expr
case class Div(e1:Expr, e2:Expr) extends Expr
case class Neg(e1:Expr) extends Expr
// ... many comparisons and such ...
case class IfThenElse(e1:Expr, e2:Expr, e3:Expr) extends Expr
case class FunCall(e1:Expr, e2:Expr) extends Expr
// Expr -> Let(Identifier, Type, Expr, Expr)
???
// Expr -> LetRec(Identifier, Type, Identifier, Type, Expr, Expr)
???

case class Const(n:Number) extends Value
case object True extends Value
case object False extends Value
// Value -> FunDef(Identifier, Type, Expr)
???

// Type -> NumType
???
// Type -> BoolType
???
// Type -> FunType(Type, Type)
???


// new thing
// handles type errors in the code
// NOTE that the type error is not itself a memeber of Type
case class TypeErrorException(s: String) extends Exception


// typeof
def typeOf(e: Expr, tenv: TypeEnv = Map()): Type = {

    // helpers here
    // create as many as you need as you need them
   
    e match {
        case Const(n) => ???  // num
        case True | False => ???  // bool
        case FunDef(x, t, e1) => ???  // type => type
        
        case Neg(e1) => ???
        case Plus(e1,e2) => ???
        case Minus(e1,e2) => ???
        case Times(e1,e2) => ???
        case Div(e1,e2) => ???
        
        case IfThenElse(e1,e2,e3) => ???
        
        case Let(x0,t0,e1,e2) => ???
        case Ident(x) => ???
        
        case LetRec(f, ft, x, xt, e1, e2) => ???
        case FunCall(e1,e2) => ???
    }
}


// test cases
def typeCheck(e:Expr, ot:Option[Type]) = {
    val b = ot match {
        case None => typeFailureCheck(e)
        case Some(t) => typeSuccessCheck(e,t)
    }
    if (b) {
        println("success!!")
    } else {
        println("FAILURE (no idea why) see above line for some details")
    }
}

def typeSuccessCheck(e:Expr, t:Type):Boolean = {
    try {
        val tGot = typeOf(e)
        tGot == t
    } catch {
        case e:Throwable => {
            println(e)
            false
        }
    }

}

def typeFailureCheck(e:Expr):Boolean = {
    try {
        typeOf(e)
        false
    } catch {
        case e:Throwable => true
    }
}

// -5
typeCheck(Neg(Const(5.0)), Some(NumType))
// 5 + 6
typeCheck(Plus(Const(5.0),Const(6.0)), Some(NumType))
// - true
typeCheck(Neg(True), None)
// true + 5
typeCheck(Plus(True,Const(5.0)), None)
// if (true) then 5 else 6
typeCheck(IfThenElse(True,Const(5.0),Const(6.0)), Some(NumType))
// if (true) then false else 6
typeCheck(IfThenElse(True,False,Const(6.0)), None)
// let f : num => num = function(x : num) x + 2 in f(5)
typeCheck(Let("f",FunType(NumType,NumType),FunDef("x",NumType,Plus(Ident("x"),Const(2.0))),FunCall(Ident("f"),Const(5.0))), Some(NumType))


cmd15.sc:56: constructor cannot be instantiated to expected type;
 found   : cmd15.this.cmd14.Let
 required: Helper.this.Expr
        case Let(x0,t0,e1,e2) => ???
             ^cmd15.sc:57: constructor cannot be instantiated to expected type;
 found   : cmd15.this.cmd14.LetRec
 required: Helper.this.Expr
        case LetRec(f, ft, x, xt, e1, e2) => ???
             ^Compilation Failed

: 

## Static vs Dynamic Typing
* As discussed there are two big options for type checking:
    * Static Typing: type is checked in static time / at compile time / before evaluation
        * C/C++
        * Java
        * Scala
        * TypeScript (JavaScript with type annotations and some other special things)
        * (Python 3.6 and above have some static type checking support)
    * Dynamic Typing: type is checked durring evaluation
        * Python
        * JavaScript
* And we've worked with type checking explicit types in statically typed lettuce
* but you may note that some of the time, noting types is sort of a pain and seems useless
    * consider lettuce expressions:
        * let x : num = 2 in x + 3
        * let f : num => num = function (x : num) x + 2 in f(3)
    * consider Scala expressions:

In [1]:
val x: Int = 2
def f(x:Int): Int = x + 2

x: Int = 2
defined function f

* Wouldn't it be nice if we didn't have to state the types of identifiers if the type is "obvious" to the programmer?
* I think it would be nice!
* We can introduce type inference to the langauge, so that in **static time**
    * the interpreter can guess what the inteded type is
    * and not bother the developer with stating the type explicitly
* lettuce
    * let x = 2
        * the interpreter assumes x is a num
    * let f = function (x) x + 2 in f(3)
        * the interpreter assumes f is a function with type num => num
        * the interpreter assumes x is a num
* scala

In [2]:
val x = 2
def f(x:Int) = x + 2

x: Int = 2
defined function f

* But how do we implement this type inference?
* How do we get the interpreter to guess the type of an expression?
* How do we get static type checking without forcing the programmer to denote the type of everything?

## Type Inference
* We shouldn't always need to state the types of expressions
* Computers are powerful
* If I as a human know the type by looking at it, surely the computer can also figure out the type
    * Maybe there is a fancy algorithm out there that will figure it out for us

### Examples
* let us try to guess the type of expressions in the following program (using the powerful human mind)

#### ex 1
* let a : ??? = 5 in a + 2
* **a** must be a ____
* the expression has type ____
    * because ______

#### ex 2
* let b : ??? = ! true in b || !b
* **b** must be a ______
* the expression has type ____
    * because ______

#### ex 3
* let c : ??? = function (d : ???) d * 4 in c(6)
* **c** must be a _______
* **d** must be a _______
* the expression has type ____
    * because ______

#### ex 4
* let f : ??? = [ function (x: ???) [ function(y : ???) x + y ]] in f(2)(3)
* f must be a ____
* x must be a ____
* y must be a ____
* the expression has type ___
    * because ____

#### ex 5
* this one demonstrates an error case
* $$\begin{array}& \\ & let\ x\ :\ ???\ & =\ 25 \\ & & in \\ && let\ f\ :\ ???\ & =\ function\ (t\ :\ ???)\ t\ >=\ 35 \\ &&& in \\ &&& f(45)\ >=\ x\end{array}$$
* **x** must be a ______
* **f** must be a ______
* **t** must be a ______
* the expression has no type because it throws a type error
    * type error is thrown because ______
        

#### ex 6
* (from Sriram's notes)
* This might have a type error
* Let us take a more complex example

~~~
let f : ? = function (g: ?) 
               function (x: ?) 
                   g( g(x) )
       in 
   let double: ? = function (x) 2 * x in 
      let doubool: ? = function (b) b and b in 
         let y = f(double)(20)  in 
           let z = f(doubool) (false) in 
               y
~~~

* f: _____
* g: _____
* x: _____
* double: _____
* doubool: _____
* y: _____
* z: _____

<br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/>

~~~
let f : ? = function (g: ?) 
               function (x: ?) 
                   g( g(x) )
       in 
   let double: ? = function (x) 2 * x in 
      let doubool: ? = function (b) b and b in 
         let y = f(double)(20)  in 
           let z = f(doubool) (false) in 
               y
~~~

This turns out to have a type error. The issue is that function `f` is called in 
two ways. In one its argument `g` has type `num => num` and `x` has type `num`. However, in the other
call argument `g` has the type `bool => bool` and `x` has type `bool`.

Therefore, function `f` and its parameters `g`, `x` are not going to receive types.

However, note that this program executes fine under our old dynamically typed interpreter. 
The type of f is _polymorphic_: (t => t) => t => t, where t can be num or bool. However,
our system does not (yet) allow polymorphic functions.
$$\newcommand\num{\mathbf{num}}$$
$$\newcommand\bool{\mathbf{bool}}$$
$$\newcommand\Ra{\Rightarrow}$$

## Unification
* There are many ways to implement this
* one is know as **Unification**
* The concept is somewhat simple:
    * generate a table of expressions to types or type variables (stand-ins for types that must be infered)
    * generate a table of sources to formulas, this will make use of the first table
    * solve the formulas
        * use lists of formulas and substitution rules
        * If this produces a logic error, then we know that the program is not well-typed
    * return a known type or the found type of a solved formula
* Today we'll look at this from a high level and not bother ourselves with how such a thing is programmed

### Expression to type variable
* observe the expression
* create a table of all subexpressions
    * I prefer fine granularity
    * but you might prefer less granularity

#### Example
EXPRESSION
~~~
let f : ??? = [
    function (x: ???) [
        function(y : ???) x + y
    ]
] in f(2)(3)
~~~

##### Tables

###### First table
* shows the full expression and its sub-expressions
<table>
<tr><th>Expression</th><th>Type/Type Var</th></tr>
<tr><td>let f : ??? = [ function (x: ???) [ function(y : ???) x + y ]] in f(2)(3)</td><td>???</td></tr>
<tr><td>f</td><td>???</td></tr>
<tr><td>function (x: ???) [ function(y : ???) x + y ]</td><td>???</td></tr>
<tr><td>f(2)(3)</td><td>???</td></tr>
</table>


###### Next tables
* do the sub-expression have more sub-expressions?
* make rows for them
<table>
<tr><th>Expression</th><th>Type/Type Var</th></tr>
<tr><td>let f : ??? = [ function (x: ???) [ function(y : ???) x + y ]] in f(2)(3)</td><td>???</td></tr>
<tr><td>f</td><td>???</td></tr>
<tr><td>function (x: ???) [ function(y : ???) x + y ]</td><td>???</td></tr>
<tr><td>x</td><td>???</td></tr>
<tr><td>function(y : ???) x + y </td><td>???</td></tr>
<tr><td>f(2)(3)</td><td>???</td></tr>
</table>

<table>
<tr><th>Expression</th><th>Type/Type Var</th></tr>
<tr><td>let f : ??? = [ function (x: ???) [ function(y : ???) x + y ]] in f(2)(3)</td><td>???</td></tr>
<tr><td>f</td><td>???</td></tr>
<tr><td>function (x: ???) [ function(y : ???) x + y ]</td><td>???</td></tr>
<tr><td>x</td><td>???</td></tr>
<tr><td>function(y : ???) x + y </td><td>???</td></tr>
<tr><td>y</td><td>???</td></tr>
<tr><td>x + y </td><td>???</td></tr>
<tr><td>f(2)(3)</td><td>???</td></tr>
</table>

<table>
<tr><th>Expression</th><th>Type/Type Var</th></tr>
<tr><td>let f : ??? = [ function (x: ???) [ function(y : ???) x + y ]] in f(2)(3)</td><td>???</td></tr>
<tr><td>f</td><td>???</td></tr>
<tr><td>function (x: ???) [ function(y : ???) x + y ]</td><td>???</td></tr>
<tr><td>x</td><td>???</td></tr>
<tr><td>function(y : ???) x + y </td><td>???</td></tr>
<tr><td>y</td><td>???</td></tr>
<tr><td>x + y</td><td>???</td></tr>
<tr><td>x</td><td>???</td></tr>
<tr><td>y</td><td>???</td></tr>
<tr><td>f(2)(3)</td><td>???</td></tr>
</table>

<table>
<tr><th>Expression</th><th>Type/Type Var</th></tr>
<tr><td>let f : ??? = [ function (x: ???) [ function(y : ???) x + y ]] in f(2)(3)</td><td>???</td></tr>
<tr><td>f</td><td>???</td></tr>
<tr><td>function (x: ???) [ function(y : ???) x + y ]</td><td>???</td></tr>
<tr><td>x</td><td>???</td></tr>
<tr><td>function(y : ???) x + y </td><td>???</td></tr>
<tr><td>y</td><td>???</td></tr>
<tr><td>x + y</td><td>???</td></tr>
<tr><td>x</td><td>???</td></tr>
<tr><td>y</td><td>???</td></tr>
<tr><td>f(2)(3)</td><td>???</td></tr>
<tr><td>f(2)</td><td>???</td></tr>
<tr><td>3</td><td>???</td></tr>
</table>

<table>
<tr><th>Expression</th><th>Type/Type Var</th></tr>
<tr><td>let f : ??? = [ function (x: ???) [ function(y : ???) x + y ]] in f(2)(3)</td><td>???</td></tr>
<tr><td>f</td><td>???</td></tr>
<tr><td>function (x: ???) [ function(y : ???) x + y ]</td><td>???</td></tr>
<tr><td>x</td><td>???</td></tr>
<tr><td>function(y : ???) x + y </td><td>???</td></tr>
<tr><td>y</td><td>???</td></tr>
<tr><td>x + y</td><td>???</td></tr>
<tr><td>x</td><td>???</td></tr>
<tr><td>y</td><td>???</td></tr>
<tr><td>f(2)(3)</td><td>???</td></tr>
<tr><td>f(2)</td><td>???</td></tr>
<tr><td>f</td><td>???</td></tr>
<tr><td>2</td><td>???</td></tr>
<tr><td>3</td><td>???</td></tr>
</table>

* NOTE: this is a bit absurd for a human to do... But a computer, while powerful, is not particularly intelligent and would likely need to create something like this
* Let's remove redundant expressions

<table>
<tr><th>Expression</th><th>Type/Type Var</th></tr>
<tr><td>let f : ??? = [ function (x: ???) [ function(y : ???) x + y ]] in f(2)(3)</td><td>???</td></tr>
<tr><td>f</td><td>???</td></tr>
<tr><td>function (x: ???) [ function(y : ???) x + y ]</td><td>???</td></tr>
<tr><td>x</td><td>???</td></tr>
<tr><td>function(y : ???) x + y </td><td>???</td></tr>
<tr><td>y</td><td>???</td></tr>
<tr><td>x + y</td><td>???</td></tr>
<tr><td>f(2)(3)</td><td>???</td></tr>
<tr><td>f(2)</td><td>???</td></tr>
<tr><td>2</td><td>???</td></tr>
<tr><td>3</td><td>???</td></tr>
</table>

* We need to add known types where possible
* numbers like 2 and 3 have type num
* the + operation always returns num in lettuce
    * note, our plus expression here is x + y
    * that might not be a num type
    * that might be an error
    * but that is okay... the Unification algorithm will handle that issue for us later on

<table>
<tr><th>Expression</th><th>Type/Type Var</th></tr>
<tr><td>let f : ??? = [ function (x: ???) [ function(y : ???) x + y ]] in f(2)(3)</td><td>???</td></tr>
<tr><td>f</td><td>???</td></tr>
<tr><td>function (x: ???) [ function(y : ???) x + y ]</td><td>???</td></tr>
<tr><td>x</td><td>???</td></tr>
<tr><td>function(y : ???) x + y </td><td>???</td></tr>
<tr><td>y</td><td>???</td></tr>
<tr><td>x + y</td><td>num</td></tr>
<tr><td>f(2)(3)</td><td>???</td></tr>
<tr><td>f(2)</td><td>???</td></tr>
<tr><td>2</td><td>num</td></tr>
<tr><td>3</td><td>num</td></tr>
</table>

* We need to create type variables for the things we do not know
* a symantic name might be useful for us as people to read this table, but some of these don't have logical names
* You might think about how the heck a computer would name these
* I am going to risk it, and use the computer method
    * start at t0
    * move to tn
    * such that n is the number of type variables already made

<table>
<tr><th>Expression</th><th>Type/Type Var</th></tr>
<tr><td>let f : ??? = [ function (x: ???) [ function(y : ???) x + y ]] in f(2)(3)</td><td>t0</td></tr>
<tr><td>f</td><td>t1</td></tr>
<tr><td>function (x: ???) [ function(y : ???) x + y ]</td><td>t2</td></tr>
<tr><td>x</td><td>t3</td></tr>
<tr><td>function(y : ???) x + y </td><td>t4</td></tr>
<tr><td>y</td><td>t5</td></tr>
<tr><td>x + y</td><td>num</td></tr>
<tr><td>f(2)(3)</td><td>t6</td></tr>
<tr><td>f(2)</td><td>t7</td></tr>
<tr><td>2</td><td>num</td></tr>
<tr><td>3</td><td>num</td></tr>
</table>

### Observations
* and just like that we have our table...
    * perhaps not the easiest thing ever
    * perhaps it won't be the simplest code that you've ever written
    * but hopefully I've demonstrated how programmatic this process could be
* I performed a breadth first search of the AST to construct the expressions of our table
    * perhaps depth first would be better
* After I created the table I removed redundant expressions
    * Implementing this in Scala, I imagine using a Map[Expression, TypeVarThingIDKWhatYet]
    * If I have a Map of key-value pairs and the keys are expressions, then the code will automatically remove redundancies for me (WOOT WOOT!)
* After creating the keys of the table I went back in and gave known types and then some type variables
    * perhaps when coding, I'll need to determine a known type or a type var on the fly, while finding my keys

### Qs?

### Try it yourself
* take the following expression and complete the table with type variables

~~~
let z : ? = 3 in
let f : ? = function (x : ?) x < 2 in 
f(z)
~~~

<table>
<tr><th>Expression</th><th>Type/Type Var</th></tr>

<tr>
    <td>
        ???
    </td>
    <td>
        ???
    </td>
</tr>
<tr>
    <td>
        ???
    </td>
    <td>
        ???
    </td>
</tr>
<tr>
    <td>
        ???
    </td>
    <td>
        ???
    </td>
</tr>
<tr>
    <td>
        ???
    </td>
    <td>
        ???
    </td>
</tr>
<tr>
    <td>
        ???
    </td>
    <td>
        ???
    </td>
</tr>
<tr>
    <td>
        ???
    </td>
    <td>
        ???
    </td>
</tr>
<tr>
    <td>
        ???
    </td>
    <td>
        ???
    </td>
</tr>
<tr>
    <td>
        ???
    </td>
    <td>
        ???
    </td>
</tr>
<tr>
    <td>
        ???
    </td>
    <td>
        ???
    </td>
</tr>
<tr>
    <td>
        ???
    </td>
    <td>
        ???
    </td>
</tr>
<tr>
    <td>
        ???
    </td>
    <td>
        ???
    </td>
</tr>
<tr>
    <td>
        ???
    </td>
    <td>
        ???
    </td>
</tr>
<tr>
    <td>
        ???
    </td>
    <td>
        ???
    </td>
</tr>
<tr>
    <td>
        ???
    </td>
    <td>
        ???
    </td>
</tr>
</table>

### Type variable to formula

* now for the formulas
* The next step would be determining formulas
* We'll look at this next week
* but here is what will happen

#### Example
TABLE from above
<table>
<tr><th>Expression</th><th>Type/Type Var</th></tr>

<tr><td>let f : ??? = [ function (x: ???) [ function(y : ???) x + y ]] in f(2)(3)</td><td>t0</td></tr>
<tr><td>f</td><td>t1</td></tr>
<tr><td>function (x: ???) [ function(y : ???) x + y ]</td><td>t2</td></tr>
<tr><td>x</td><td>t3</td></tr>
<tr><td>function(y : ???) x + y </td><td>t4</td></tr>
<tr><td>y</td><td>t5</td></tr>
<tr><td>x + y</td><td>num</td></tr>
<tr><td>f(2)(3)</td><td>t6</td></tr>
<tr><td>f(2)</td><td>t7</td></tr>
<tr><td>2</td><td>num</td></tr>
<tr><td>3</td><td>num</td></tr>
</table>

We will construct the following table of equations
I'll show you how next week

<table>
<tr><th>Source</th><th>Equation</th></tr>

<tr><td>let f : ??? = [ function (x: ???) [ function(y : ???) x + y ]] in f(2)(3)</td><td>t0 = t6</td></tr>
<tr><td>f</td><td>t1 = t2</td></tr>
<tr><td>function (x: ???) [ function(y : ???) x + y ]</td><td>t2 = (t3 => t4)</td></tr>
<tr><td>x</td><td>t3 = num</td></tr>
<tr><td>function(y : ???) x + y </td><td>t4 = (t5 => num)</td></tr>
<tr><td>y</td><td>t5 = num</td></tr>
<tr><td>f(2)(3)</td><td>t6 = num</td></tr>
<tr><td>f(2)</td><td>t7 = t4</td></tr>
</table>

#### Observations
* let x = e_bind in e_body
    * denote with type variable t 
    * t = t_e_body
* function (x) e_body
    * denote with type variable t
    * t = (t_x => t_e_body)
* e_func(e_arg)
    * denote with type variable t
    * t_e_func = (t_arg => t_ret)
    * t = t_ret

### Substitutions
* Next we need to discover our substitutions from our equations
* There are a few properties that must hold true here (stolen from Sriram's notes)
    - **P1.** There can be at most one rule of the form $t \mapsto ...$ for each type variable $t$.
    - **P2.** If $t$ appears in the LHS of some substitution rule, then it cannot appear in the RHS of any substitution rule.
    - **P3.** Trivial substitutions such as $t \mapsto t$ should be discarded.
    - **P4.** If a rule involves $t$ on both LHS and RHS, then we declare TYPE ERROR.
    - **P5.** If one side of the Eq. is $\num, \bool$ and the other side is not the same, then we immediately conclude __type error__.
    - **P6.** If both sides are funtion types $ (t_1 \Ra t_2) =  (t_3 \Ra t_4)$, we add the equations
$$ t_1 = t_3\ \text{and}\ t_2 = t_4 $$.

* We will construct demostrate that the table can be used to demonstrate that the type of the program is infact **num**
* I'll show you why next week

<table>
<tr><th>Source</th><th>Equation</th></tr>

<tr><td>let f : ??? = [ function (x: ???) [ function(y : ???) x + y ]] in f(2)(3)</td><td>t0 = t6</td></tr>
<tr><td>f</td><td>t1 = t2</td></tr>
<tr><td>function (x: ???) [ function(y : ???) x + y ]</td><td>t2 = (t3 => t4)</td></tr>
<tr><td>x</td><td>t3 = num</td></tr>
<tr><td>function(y : ???) x + y </td><td>t4 = (t5 => num)</td></tr>
<tr><td>y</td><td>t5 = num</td></tr>
<tr><td>f(2)(3)</td><td>t6 = num</td></tr>
<tr><td>f(2)</td><td>t7 = t4</td></tr>
</table>


* substitution rules found
    * t0 = num
    * t1 = (num => (num => num))
    * t2 = (num => (num => num))
    * t3 = num
    * t4 = (num => num)
    * t5 = num
    * t6 = num
    * t7 = (num => num)


### Observations
* None quite yet. I am still mastering this on my own and trying not too look it all up until I've tried it myself. :(

### Qs???

## Solutions
* Please don't look at these until after class

### typeOf first solution

In [18]:
// AST
sealed trait Expr
sealed trait Value extends Expr
sealed trait Type
type Identifier = String
type Number = Double
type TypeEnv = Map[Identifier,Type]

case class Ident(x:Identifier) extends Expr
case class Plus(e1:Expr, e2:Expr) extends Expr
case class Minus(e1:Expr, e2:Expr) extends Expr
case class Times(e1:Expr, e2:Expr) extends Expr
case class Div(e1:Expr, e2:Expr) extends Expr
case class Neg(e1:Expr) extends Expr
case class Or(e1:Expr, e2:Expr) extends Expr
case class And(e1:Expr, e2:Expr) extends Expr
case class Not(e1:Expr) extends Expr
case class IfThenElse(e1:Expr, e2:Expr, e3:Expr) extends Expr
case class Let(x:Identifier, t:Type, eBind:Expr, eBody:Expr) extends Expr
case class LetRec(f:Identifier, ft:Type, x:Identifier, xt:Type, eBind:Expr, eBody:Expr) extends Expr
case class FunCall(e1:Expr, e2:Expr) extends Expr

case class Const(n:Number) extends Value
case object True extends Value
case object False extends Value
case class FunDef(x:Identifier, t:Type, e:Expr) extends Value

case object NumType extends Type
case object BoolType extends Type
case class FunType(t1:Type, t2:Type) extends Type

case class TypeErrorException(s: String) extends Exception


// typeof
def typeOf(e: Expr, tenv: TypeEnv = Map()): Type = {

    def typ(e:Expr):Type = typeOf(e,tenv)
    
    def binArith(e1:Expr, e2:Expr): Type = {
        typ(e1) match {
            case NumType => typ(e2) match {
                case NumType => NumType
                case tgot => throw new TypeErrorException(s"Type mismatch: $tgot found, expected NumType")
            }
            case tgot => throw new TypeErrorException(s"Type mismatch: $tgot found, expected NumType")
        }
    }
    
    def binLogic(e1:Expr, e2:Expr): Type = {
        typ(e1) match {
            case BoolType => typ(e2) match {
                case BoolType => BoolType
                case tgot => throw new TypeErrorException(s"Type mismatch: $tgot found, expected BoolType")
            }
            case tgot => throw new TypeErrorException(s"Type mismatch: $tgot found, expected BoolType")
        }
    }
    
    e match {
        case Const(n) => NumType
        case True | False => BoolType
        case Ident(x) => tenv.getOrElse(x, throw new TypeErrorException(s"Type mismatch: $x not defined"))
        case FunDef(x,t,e) => FunType(t, typeOf(e,tenv + (x -> t)))
        case Plus(e1,e2) => binArith(e1,e2)
        case Minus(e1,e2) => binArith(e1,e2)
        case Times(e1,e2) => binArith(e1,e2)
        case Div(e1,e2) => binArith(e1,e2)
        case Neg(e1) => typ(e1) match {
            case NumType => NumType
            case tgot => throw new TypeErrorException(s"Type mismatch: $tgot found, expected NumType")
        }
        case And(e1,e2) => binLogic(e1,e2)
        case Or(e1,e2) => binLogic(e1,e2)
        case Not(e1) => typ(e1) match {
            case BoolType => BoolType
            case tgot => throw new TypeErrorException(s"Type mismatch: $tgot found, expected BoolType")
        }
        case IfThenElse(e1,e2,e3) => typ(e1) match {
            case BoolType => {
                val t2 = typ(e2)
                val t3 = typ(e3)
                if (t2 == t3) {
                    t2
                } else {
                    throw new TypeErrorException(s"Type mismatch: $e3 has type $t3, expected $t2")
                }
            }
            case tgot => throw new TypeErrorException(s"Type mismatch: $tgot found, expected BoolType") 
        }
        case FunCall(e1,e2) => typ(e1) match {
            case FunType(inT, outT) => {
                val t2 = typ(e2)
                if (inT == t2) {
                    outT
                } else {
                    throw new TypeErrorException(s"Type mismatch: expecteed argument type $inT, found $t2")
                }
            }
            case tgot => throw new TypeErrorException(s"Type mismatch: expected function type for $e1 in parent expression $e, found $tgot)")
        }
        case Let(x0,t0,e1,e2) => {
            val t1 = typ(e1)
            if (t0 == t1) {
                val tenvp = tenv + (x0 -> t0)
                typeOf(e2,tenvp)
            } else {
                throw new TypeErrorException(s"Type mismatch: $x0 should have type $t0, but found $t1")
            }
        }
        case LetRec(f, ft, x, xt, e1, e2) => {
            ft match {
                case FunType(inputType, retType) => {
                    if (inputType == xt) {
                        val tenvp = tenv + (f -> ft) + (x -> xt)
                        val t1 = typeOf(e1,tenvp)
                        if (t1 == retType) {
                            // interesting to note here.... x -> xt doesn't get carried forward
                            // there is a logical reason
                            // I encourage you to think about it
                            typeOf(e2, tenv + (f -> ft))
                        } else {
                            throw new TypeErrorException(s"Epected return type $retType, but found $t1 in $e")
                        }
                    } else {
                        throw new TypeErrorException(s"Expected function input type $inputType, but found $xt in $e")
                    }
                }
                case tgot => throw new TypeErrorException(s"Type mismatch: Expected FunType for $ft in $e, but found $tgot")
            }
        }
    }
}


// test cases
def typeCheck(e:Expr, ot:Option[Type]) = {
    val b = ot match {
        case None => typeFailureCheck(e)
        case Some(t) => typeSuccessCheck(e,t)
    }
    if (b) {
        println("success!!")
    } else {
        println("FAILURE (no idea why) see above line for some details")
    }
}

def typeSuccessCheck(e:Expr, t:Type):Boolean = {
    try {
        val tGot = typeOf(e)
        tGot == t
    } catch {
        case e:Throwable => {
            println(e)
            false
        }
    }

}

def typeFailureCheck(e:Expr):Boolean = {
    try {
        typeOf(e)
        false
    } catch {
        case e:Throwable => true
    }
}

// -5
typeCheck(Neg(Const(5.0)), Some(NumType))
// 5 + 6
typeCheck(Plus(Const(5.0),Const(6.0)), Some(NumType))
// - true
typeCheck(Neg(True), None)
// true + 5
typeCheck(Plus(True,Const(5.0)), None)
// if (true) then 5 else 6
typeCheck(IfThenElse(True,Const(5.0),Const(6.0)), Some(NumType))
// if (true) then false else 6
typeCheck(IfThenElse(True,False,Const(6.0)), None)
// let f : num => num = function(x : num) x + 2 in f(5)
typeCheck(Let("f",FunType(NumType,NumType),FunDef("x",NumType,Plus(Ident("x"),Const(2.0))),FunCall(Ident("f"),Const(5.0))), Some(NumType))


success!!
success!!
success!!
success!!
success!!
success!!
success!!


defined trait Expr
defined trait Value
defined trait Type
defined type Identifier
defined type Number
defined type TypeEnv
defined class Ident
defined class Plus
defined class Minus
defined class Times
defined class Div
defined class Neg
defined class Or
defined class And
defined class Not
defined class IfThenElse
defined class Let
defined class LetRec
defined class FunCall
defined class Const
defined object True
defined object False
defined class FunDef
defined object NumType
defined object BoolType
defined class FunType
defined class TypeErrorException
defined function typeOf
defined function typeCheck
defined function typeSuccessCheck
defined function typeFailureCheck

### typeOf second solution
* the above solution is messy
* I figure I could clean it up
* **NOTE**: this solution makes use of **side effects**
    * namely the functions typEq and typsEq might throw an error
    * if an error is thrown the rest of the expresssion is not evaluated

In [20]:
// AST
sealed trait Expr
sealed trait Value extends Expr
sealed trait Type
type Identifier = String
type Number = Double
type TypeEnv = Map[Identifier,Type]

case class Ident(x:Identifier) extends Expr
case class Plus(e1:Expr, e2:Expr) extends Expr
case class Minus(e1:Expr, e2:Expr) extends Expr
case class Times(e1:Expr, e2:Expr) extends Expr
case class Div(e1:Expr, e2:Expr) extends Expr
case class Neg(e1:Expr) extends Expr
case class Or(e1:Expr, e2:Expr) extends Expr
case class And(e1:Expr, e2:Expr) extends Expr
case class Not(e1:Expr) extends Expr
case class IfThenElse(e1:Expr, e2:Expr, e3:Expr) extends Expr
case class Let(x:Identifier, t:Type, eBind:Expr, eBody:Expr) extends Expr
case class LetRec(f:Identifier, ft:Type, x:Identifier, xt:Type, eBind:Expr, eBody:Expr) extends Expr
case class FunCall(e1:Expr, e2:Expr) extends Expr

case class Const(n:Number) extends Value
case object True extends Value
case object False extends Value
case class FunDef(x:Identifier, t:Type, e:Expr) extends Value

case object NumType extends Type
case object BoolType extends Type
case class FunType(t1:Type, t2:Type) extends Type

case class TypeErrorException(s: String) extends Exception


// typeof
def typeOf(e: Expr, tenv: TypeEnv = Map()): Type = {

    def typ(e:Expr):Type = typeOf(e,tenv)
    
    def typEq(e:Expr, t:Type): Type = {
        val tgot = typ(e)
        if (t == tgot) {
            tgot
        } else {
            throw new TypeErrorException(s"Type mismatch: expected $t, found $tgot")
        }
    }
    
    def typsEq(es:List[Expr], t:Type):Type = {
        es map { e => typEq(e,t) }
        t
    }
    
    def binArith(e1:Expr, e2:Expr): Type = typsEq(List(e1,e2), NumType)
    
    def binLogic(e1:Expr, e2:Expr): Type = typsEq(List(e1,e2), BoolType)
    
    e match {
        case Const(n) => NumType
        case True | False => BoolType
        case Ident(x) => tenv.getOrElse(x, throw new TypeErrorException(s"Type mismatch: $x not defined"))
        case FunDef(x,t,e) => FunType(t, typeOf(e,tenv + (x -> t)))
        case Plus(e1,e2) => binArith(e1,e2)
        case Minus(e1,e2) => binArith(e1,e2)
        case Times(e1,e2) => binArith(e1,e2)
        case Div(e1,e2) => binArith(e1,e2)
        case Neg(e1) => typEq(e1, NumType)
        case And(e1,e2) => binLogic(e1,e2)
        case Or(e1,e2) => binLogic(e1,e2)
        case Not(e1) => typEq(e1, BoolType)
        case IfThenElse(e1,e2,e3) => {
            typEq(e1,BoolType)
            val t2 = typ(e2)
            typEq(e3, t2)
        }
        case FunCall(e1,e2) => typ(e1) match {
            case FunType(inT, outT) => {
                typEq(e2,inT)
                outT
            }
            case tgot => throw new TypeErrorException(s"Type mismatch: $tgot found, expected BoolType") 
        }
        case Let(x0,t0,e1,e2) => {
            val t1 = typEq(e1,t0)
            val tenvp = tenv + (x0 -> t0)
            typeOf(e2,tenvp)
        }
        case LetRec(f, ft, x, xt, e1, e2) => {
            ft match {
                case FunType(inputType, retType) => {
                    if (inputType == xt) {
                        val tenvp = tenv + (f -> ft) + (x -> xt)
                        val t1 = typeOf(e1,tenvp)
                        if (t1 == retType) {
                            // interesting to note here.... x -> xt doesn't get carried forward
                            // there is a logical reason
                            // I encourage you to think about it
                            typeOf(e2, tenv + (f -> ft))
                        } else {
                            throw new TypeErrorException(s"Epected return type $retType, but found $t1 in $e")
                        }
                    } else {
                        throw new TypeErrorException(s"Expected function input type $inputType, but found $xt in $e")
                    }
                }
                case tgot => throw new TypeErrorException(s"Type mismatch: Expected FunType for $ft in $e, but found $tgot")
            }
        }
    }
}



// test cases
def typeCheck(e:Expr, ot:Option[Type]) = {
    val b = ot match {
        case None => typeFailureCheck(e)
        case Some(t) => typeSuccessCheck(e,t)
    }
    if (b) {
        println("success!!")
    } else {
        println("FAILURE (no idea why) see above line for some details")
    }
}

def typeSuccessCheck(e:Expr, t:Type):Boolean = {
    try {
        val tGot = typeOf(e)
        tGot == t
    } catch {
        case e:Throwable => {
            println(e)
            false
        }
    }

}

def typeFailureCheck(e:Expr):Boolean = {
    try {
        typeOf(e)
        false
    } catch {
        case e:Throwable => true
    }
}

// -5
typeCheck(Neg(Const(5.0)), Some(NumType))
// 5 + 6
typeCheck(Plus(Const(5.0),Const(6.0)), Some(NumType))
// - true
typeCheck(Neg(True), None)
// true + 5
typeCheck(Plus(True,Const(5.0)), None)
// if (true) then 5 else 6
typeCheck(IfThenElse(True,Const(5.0),Const(6.0)), Some(NumType))
// if (true) then false else 6
typeCheck(IfThenElse(True,False,Const(6.0)), None)
// let f : num => num = function(x : num) x + 2 in f(5)
typeCheck(Let("f",FunType(NumType,NumType),FunDef("x",NumType,Plus(Ident("x"),Const(2.0))),FunCall(Ident("f"),Const(5.0))), Some(NumType))


success!!
success!!
success!!
success!!
success!!
success!!
success!!


defined trait Expr
defined trait Value
defined trait Type
defined type Identifier
defined type Number
defined type TypeEnv
defined class Ident
defined class Plus
defined class Minus
defined class Times
defined class Div
defined class Neg
defined class Or
defined class And
defined class Not
defined class IfThenElse
defined class Let
defined class LetRec
defined class FunCall
defined class Const
defined object True
defined object False
defined class FunDef
defined object NumType
defined object BoolType
defined class FunType
defined class TypeErrorException
defined function typeOf
defined function typeCheck
defined function typeSuccessCheck
defined function typeFailureCheck

### TypeOf with different AST
* Note the IdentDef type and how it is used in the AST
* I think this would be fairly extensible
* **NOTE**: this solution makes use of **side effects**
    * namely the functions typEq might throw an error
    * if an error is thrown the rest of the expresssion is not evaluated

In [15]:
// AST
sealed trait Expr
sealed trait Value extends Expr
sealed trait Type
type Identifier = String
type IdentDef = (String, Type)
type Number = Double
type TypeEnv = Map[Identifier, Type]

case class Ident(x:Identifier) extends Expr
case class Plus(e1:Expr, e2:Expr) extends Expr
case class Neg(e1:Expr) extends Expr
case class Let(x:IdentDef, eBind:Expr, eBody:Expr) extends Expr
case class LetRec(f:IdentDef, x:IdentDef, eBind:Expr, eBody:Expr) extends Expr
case class FunCall(e1:Expr, e2:Expr) extends Expr

case class Const(n:Number) extends Value
case object True extends Value
case object False extends Value
case class FunDef(x:IdentDef, e:Expr) extends Value

case object NumType extends Type
case object BoolType extends Type
case class FunType(t1:Type, t2:Type) extends Type

case class TypeErrorException(s: String) extends Exception


// typeof
def typeOf(e: Expr, tenv: TypeEnv = Map()): Type = {
    
    def typ(e:Expr):Type = typeOf(e)
    
    def tEq(e:Expr, t:Type):Type = {
        if (typ(e) == t) t
        else ???  // throw an error if the type is not acceptable
    }
    
    e match {
        case Const(n) => NumType
        case True | False => BoolType
        case Ident(x) => tenv.getOrElse(x, ???)  // throw some error if x no in environment
        case Neg(e1) => tEq(e1, NumType)
        case Plus(e1,e2) => {
            List(e1,e2) map { e => tEq(e, NumType) }
            NumType
        }
        case Let((x,t),e1,e2) => {
            tEq(e1,t)
            typeOf(e2, tenv + (x -> t))
        }
        case _ => ??? // not yet implemented
    }
}


defined trait Expr
defined trait Value
defined trait Type
defined type Identifier
defined type IdentDef
defined type Number
defined type TypeEnv
defined class Ident
defined class Plus
defined class Minus
defined class Times
defined class Div
defined class Neg
defined class Or
defined class And
defined class Not
defined class IfThenElse
defined class Let
defined class LetRec
defined class FunCall
defined class Const
defined object True
defined object False
defined class FunDef
defined object NumType
defined object BoolType
defined class FunType
defined class TypeErrorException
defined function typeOf

### Type Variables 
* take the following expression and complete the type/type var table 

~~~
let z : ? = 3 in
let f : ? = function (x : ?) x < 2 in 
f(z)
~~~

<table>
<tr><th>Expression</th><th>Type/Type Var</th></tr>
<tr><td>let z : ? = 3 in [ let f : ? =  [ function (x : ?) x < 2 ] in f(z) ]</td><td>t0</td></tr>
<tr><td>z</td><td>t1</td></tr>
<tr><td>3</td><td>num</td></tr>
<tr><td>let f : ? =  [ function (x : ?) x < 2 ] in f(z)</td><td>t2</td></tr>
<tr><td>f</td><td>t3</td></tr>
<tr><td>function (x : ?) x < 2</td><td>t4</td></tr>
<tr><td>x</td><td>t5</td></tr>
<tr><td>x < 2</td><td>bool</td></tr>
<tr><td>2</td><td>num</td></tr>
<tr><td>f(z)</td><td>t6</td></tr>
</table>

## Overview
* Explicit Type Checking (static time)
* Review Static vs Dynamic Type Checking
* Type Inference (still static time)
* Unification

## TODOs:
* Project 2
    * Now due this Friday, April, 5, 2019 just before midnight
    * If you haven't started, please start ASAP
        * I don't expect the project to take too long, but you just never know... better safe than sorry
* Spot Exam 3
    * Not this Friday's Recitation! WOO! It's on 04/12
    * This will be a Moodle exam (like spot exam 2)
    * This will cover:
        * Scala Higher Order methods of Lists (map, reduce, fold...)
        * Lettuce recursive functions
            * value of a program
            * use of ExtendRec
        * References and Mutability
            * Implicit
            * Explicit
            * (sideeffects)
        * CPS transforms using the _k notation and generic types
* We are realeasing a homework soon. It will probably be due Tuesday 04/16 (after the exam)